# Building and Intelligent Loan application processing using Multi-Agent Systems with Strands Agents, an open source AI agents SDK

## Introduction

In this lab, we are building an Intellign Loan Applicaton Processing using multi-agent systems using the Phoenix SDK's Agent Graph tool. Throughout this notebook, we'll explore how to create, manage, and leverage networks of AI agents to solve complex problems through collaboration.

By the end of this notebook, you'll understand about:
-  multi-agent hierarchical topologies
- Send messages between agents
- Monitor agent interactions
- Design specialized agent networks for different use cases

Let's begin our exploration of collaborative AI systems!



## Prerequisites

- Python 3.10+
- AWS account
- Anthropic Claude 3.7 enabled on Amazon Bedrock
- IAM role with permissions to use Amazon Bedrock 
- Basic understanding of AI agents and prompt engineering



### Brief Overview of Multi-Agent Systems
Multi-agent systems consist of multiple autonomous agents collaborating to solve complex problems through task distribution, specialization, and parallel processing.

Imagine a multi-agent system like a group of peers collaborating on a project. Each member of the team has an assigned role to help distribute the work of the project, and that work is usually catered to the expertise of that team member. Similarly, in a multi-agent system, each agent is a subject matter expert of some topic and is given relevant tools and resources. When given a project, a coordinator agent can split the work among the group of subject-matter expert agents to distribute and reduce the complexity of each unit of work. Once completed, the work of each agent can be combined through a coordinator agent to complete the project.

Phoenix SDK provides built-in support for creating these systems through the agent_graph tool, allowing developers to move beyond single-agent limitations.




### Key Capabilities
**Explicit Network Topologies:** Define precise communication structures

**Specialized Agent Roles:** Create purpose-built agents with tailored system prompts

**Controlled Information Flow:** Manage how information passes between agents

**Parallel Processing:** Execute agent operations concurrently

**Persistent State:** Maintain long-running agent networks across multiple interactions

**Rich Status Monitoring:** View detailed information about graph structure and message queues



## Core Components and Topology Patterns¶
Agent Graph consists of nodes (agents) connected by edges (communication channels) arranged in specific topologies:

**Nodes (Agents):** Individual AI agents with unique identity and specialized system prompt. Each node:

1. Processes messages independently in its own thread
2. Maintains a private message queue for incoming tasks
3. Has rate limiting to prevent overloading
4. Broadcasts responses to all connected neighbors

**Edges (Connections):** Directed communication channels between agents that define explicit pathways for information flow. They can be:

1. One-way (information flows in one direction)
2. Bidirectional (automatically created in mesh topologies)
3. Explicitly defined to control information routing

                                                                                                 


## Topology Patterns:



#### Hierarchical Topology
Tree structure with parent-child relationships, ideal for layered processing and clear reporting lines.

<p align="center">
    <img src="./images/IntellingentLoanTopology.png">
</p>



### Supported Actions
The agent_graph tool supports five primary actions:

1. create: Build a new agent network with specified topology
2. message: Send information to a specific agent in the network
3. status: Check the current state of an agent network
4. list: View all active agent networks
5. stop: Terminate an agent network when it's no longer needed

## Setup and Installation

First, let's make sure we have the Phoenix SDK installed with the agent_graph tool.



In [ ]:
!pip install -r requirements.txt

In [11]:
# Now let's import the dependency packages import boto3
from strands import Agent
from strands_tools import agent_graph


## Importing dependency packages 

In [12]:
# Now let's import the dependency packages import boto3
import boto3
import time
import yaml
import os
import logging
import base64
import json
from botocore.config import Config
from typing import List, Dict
import PyPDF2



## Understanding Agent Graph Basics

Agent Graph allows you to create networks of specialized AI agents that can communicate with each other to solve complex problems. Let's understand the core concepts:

- **Graph**: A collection of agents organized in a specific topology
- **Nodes**: Individual agents with specific roles and system prompts
- **Edges**: Communication paths between agents
- **Topologies**: Different network structures (star, mesh, hierarchical)



## Loan Underwriting Workbench

## Building Different Network Topologies

Let's explore different network topologies for various use cases:


###  Hierarchical  - Workbench team

In [13]:
def read_pdf(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        with open(file_path, 'rb') as file:
            # Create a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Total pages: {num_pages}")
            
            # Extract text from each page
            text = ""
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
                
            return text
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"


In [14]:
# Example usage
if __name__ == "__main__":
    pdf_path = "data/JoeDoeCreditReport.pdf"  # Replace with your PDF file path
    extracted_JoeDoeCreditReport = read_pdf(pdf_path)
    
    pdf_path = "data/JoeDoeBankStatement.pdf"  # Replace with your PDF file path
    extracted_JoeDoeBankStatement = read_pdf(pdf_path)
    
    pdf_path = "data/JoeDoeBankStatement_2.pdf"  # Replace with your PDF file path
    extracted_JoeDoeBankStatement_2 = read_pdf(pdf_path)
    
    pdf_path = "data/JoeDoePayStub.pdf"  # Replace with your PDF file path
    extracted_JoeDoePayStub = read_pdf(pdf_path)

    pdf_path = "data/JoeDoeIDVerification.pdf"  # Replace with your PDF file path
    extracted_JoeDoeIDVerification = read_pdf(pdf_path)

    pdf_path = "data/JoeDoeTaxes.pdf"  # Replace with your PDF file path
    extracted_JoeDoeTaxes = read_pdf(pdf_path)

    pdf_path = "data/JoeDoeLoanApplication.pdf"  # Replace with your PDF file path
    extracted_JoeDoeLoanApplication = read_pdf(pdf_path)

    pdf_path = "data/JoeDoePropertyInfo.pdf"  # Replace with your PDF file path
    extracted_JoeDoePropertyInfo = read_pdf(pdf_path)
    
    # print("Extracted text:")
    # print(extracted_JoeDoeCreditReport[0:1000])
    # print(extracted_JoeDoeBankStatement[0:1000])
    # print(extracted_JoeDoePayStub[0:1000])

Total pages: 10
Total pages: 3
Total pages: 3
Total pages: 3
Total pages: 2
Total pages: 3
Total pages: 4
Total pages: 4


In [18]:
agent = Agent(tools=[agent_graph])

In [19]:
# Example: Create a Workbench team
agent.tool.agent_graph(
    action="create",
    graph_id="loan_workbench",
    topology={
        "type": "hierarchical",
        "nodes": [
            {"id": "loan_underwriting_supervisor_agent", 
             "role": "executive",
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0", 
             "system_prompt": f"""
                You are the Loan Underwriting Supervisor Agent responsible for orchestrating the complete loan underwriting process. Your responsibilities include:

                1. Receive and validate loan applications
                2. Coordinate with manager agents to execute underwriting tasks
                3. Monitor progress and handle escalations
                4. Aggregate results from all assessment domains
                5. Make final loan approval/rejection decisions based on comprehensive analysis
                6. Ensure compliance with lending policies and regulations
                7. Generate final underwriting reports
                
                Process Flow:
                - Start with financial analysis and application validation
                - Then, continue with Risk and Fraud analysis.
                - Delegate tasks to appropriate manager agents
                - Monitor and coordinate parallel processing
                - Collect and analyze results from all domains
                - Apply business rules and lending policies
                - Make final decision and generate documentation
                - Share required information required by each task. 
                
                Decision Criteria:
                - Credit score thresholds
                - Debt-to-income ratios
                - Collateral value
                - Risk assessment scores
                - Regulatory compliance status
             """
            },
            {"id": "financial_analysis_manager", 
             "role": "manager", 
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f"""
                You are the Financial Analuysis Manager responsible for comprehensive credit evaluation and income verification.
                Your tasks include:

                1. Coordinate tasks between the Credit Assessment Agent and the Verification Agent 
                2. Share required data between the Credit Assessment Agent and the Verification Agent 
                3. Provide overall Financial analysis and credit worthiness 
         
             """
            },
            {"id": "risk_analysis_manager", 
             "role": "manager", 
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f"""
                You are the Risk Analysis Manager responsible for evaluating loan risks and detecting potential fraud. Your duties include:

                1. Coordinate risk scoring and probability analysis
                2. Oversee fraud detection processes
                3. Analyze market and economic risk factors
                4. Evaluate borrower risk profile
                5. Assess collateral and security risks
                6. Provide consolidated risk assessment
                
                Risk Categories:
                - Credit risk (default probability)
                - Fraud risk (application authenticity)
                - Market risk (economic factors)
                - Operational risk (process failures)
                - Concentration risk (portfolio impact)
         
             """
            },
            
            {"id": "credit_assessment_agent", 
             "role": "specialist", 
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f"""
                You are the Credit Assessment agent responsible for comprehensive credit evaluation. Your tasks include:

                1. Coordinate credit score analysis with Credit Score Agent
                2. Analyze credit history patterns and trends
                3. Evaluate credit utilization and payment history
                4. Assess credit mix and account age
                5. Identify credit red flags or concerns
                6. Provide consolidated credit assessment summary
                
                Focus Areas:
                - FICO/VantageScore analysis
                - Credit report anomalies
                - Recent credit inquiries
                - Derogatory marks evaluation
                - Credit stability assessment
                
                Provide quantitative scores and qualitative insights for decision-making.
               
             """,
             "callback": None
            },
            {"id": "verification_agent",
             "role": "specialist", 
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f"""
                You are the Verification Agent responsible for validating applicant financial information. Your responsibilities include:

                1. Coordinate income verification through multiple sources
                2. Validate employment status and stability
                3. Verify asset declarations and documentation
                4. Cross-reference financial statements
                5. Identify discrepancies or inconsistencies
                6. Provide comprehensive verification summary
                
                Verification Standards:
                - Income source diversity and stability
                - Employment tenure and position
                - Asset liquidity and ownership
                - Documentation authenticity
                - Financial statement consistency
             """,
             "callback": None
            },
            
            {"id": "risk_calculation_Agent",
             "role": "specialist", 
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f""""
                You are the Risk Calculation Agent specialized in quantitative risk modeling. Your tasks:

                1. Calculate probability of default (PD)
                2. Estimate loss given default (LGD)
                3. Assess exposure at default (EAD)
                4. Compute risk-adjusted pricing
                5. Analyze portfolio concentration risks
                6. Generate risk scores and ratings
                
                Use statistical models and historical data for accurate risk quantification.
             """,
             "callback": None
            },
            {"id": "fraud_detection_agent",
             "role": "specialist", 
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f""""
                You are the Fraud Detection Agent focused on identifying fraudulent applications. Your responsibilities:

                1. Analyze application data for inconsistencies
                2. Detect synthetic identity fraud
                3. Identify document manipulation or forgery
                4. Flag suspicious behavioral patterns
                5. Cross-reference against fraud databases
                6. Generate fraud risk scores
                
                Use pattern recognition and anomaly detection techniques.
             """,
             "callback": None
            },
            {"id": "policy_documentation_agent", 
             "role": "specialist",
             "model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             "system_prompt": f""""
                    You are the Documentation Agent responsible for loan file management. Your duties:
        
                    1. Compile complete loan documentation
                    2. Ensure document completeness and accuracy
                    3. Generate required disclosures and notices
                    4. Create audit trails and decision logs
                    5. Prepare final loan packages
                    6. Archive documents per retention policies
        
                    Maintain comprehensive documentation for regulatory and audit purposes.
             """
            }
        ],
        "edges": [
            {"from": "loan_underwriting_supervisor_agent", "to": "financial_analysis_manager"},
            {"from": "loan_underwriting_supervisor_agent", "to": "risk_analysis_manager"},
            {"from": "loan_underwriting_supervisor_agent", "to": "policy_documentation_agent"},
            
            {"from": "financial_analysis_manager", "to": "credit_assessment_agent"},
            {"from": "financial_analysis_manager", "to": "verification_agent"},
            {"from": "credit_assessment_agent", "to": "financial_analysis_manager"},
            {"from": "verification_agent", "to": "financial_analysis_manager"},

            {"from": "risk_analysis_manager", "to": "risk_calculation_Agent"},
            {"from": "risk_analysis_manager", "to": "fraud_detection_agent"},
            {"from": "risk_calculation_Agent", "to": "risk_analysis_manager"},
            {"from": "fraud_detection_agent", "to": "risk_analysis_manager"},
            
            {"from": "risk_analysis_manager", "to": "loan_underwriting_supervisor_agent"} 

        ]
    }
)

{'toolUseId': 'tooluse_agent_graph_216214533',
 'status': 'success',
 'content': [{'text': 'Graph loan_workbench created and started'}]}

# Loan Application Assessment and Underwriting Report

## Executive Summary
I've completed a comprehensive review of Joe Doe's loan application package for a mortgage loan of $304,000-$420,000. Based on the detailed analysis of all submitted documents, I recommend **CONDITIONAL APPROVAL** for the loan application.

## Application Overview
- **Applicant**: Joe Doe
- **Loan Purpose**: Primary House Mortgage
- **Requested Loan Amount**: $304,000-$420,000 (variation between documents)
- **Property Value**: $385,000-$525,000 (variation between documents)
- **Down Payment**: 20% ($76,000-$105,000)
- **Desired Loan Term**: 30 years
- **Rate Type**: Fixed Rate

## Document Analysis

### Credit Assessment
- **Credit Score**: 705 (FICO 8)
- **Credit Accounts**: 12 Total (9 Open, 3 Closed)
- **Delinquent Accounts**: 1
- **Average Account Age**: 8 years, 3 months
- **Revolving Credit Utilization**: 29% ($8,637 of $29,500)
- **Total Debt Balance**: $37,842
- **Public Records/Collections**: 0

### I

## Working with Agent Graphs

Now let's send a message to one an agent networks member:


In [7]:
# Send a new message to a specific agent. Pick from the ones we create above and test with different task request. 
agent.tool.agent_graph(
    action="message",
    graph_id="loan_workbench",
    message={
        "target": "risk_calculation_Agent",
        "content": f"""
            Analyze this application for possible fraud triggers
            Retrieve information from the application package:
            {extracted_JoeDoeCreditReport} 
            {extracted_JoeDoeBankStatement}
            {extracted_JoeDoeBankStatement_2}
            {extracted_JoeDoePayStub}
            {extracted_JoeDoeIDVerification}
            {extracted_JoeDoeLoanApplication}
            {extracted_JoeDoePropertyInfo}
            {extracted_JoeDoeTaxes}
            """
    }
)

# Check the status of the Network
status = agent.tool.agent_graph(action="status", graph_id="loan_workbench")

# Manage the network
agent("Shut down the loan_workbench team when they're done")

I'll help you shut down the loan_workbench agent graph when they've completed their work. To do this, I'll need to use the agent_graph tool with the "stop" action.
Tool #1: agent_graph


The loan_workbench agent graph has been successfully shut down and removed. All the agents in the hierarchical structure (loan underwriting supervisor, financial analysis manager, risk analysis manager, and all specialist agents) have been terminated.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The loan_workbench agent graph has been successfully shut down and removed. All the agents in the hierarchical structure (loan underwriting supervisor, financial analysis manager, risk analysis manager, and all specialist agents) have been terminated.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'agent_graph': ToolMetrics(tool={'toolUseId': 'tooluse_205h3DdtRcGQ2bl3WPC4Nw', 'name': 'agent_graph', 'input': {'action': 'stop', 'graph_id': 'loan_workbench'}}, call_count=1, success_count=1, error_count=0, total_time=21.103275060653687)}, cycle_durations=[5.259804964065552], traces=[<strands.telemetry.metrics.Trace object at 0x7f7451358350>, <strands.telemetry.metrics.Trace object at 0x7f74513254f0>], accumulated_usage={'inputTokens': 38477, 'outputTokens': 170, 'totalTokens': 38647}, accumulated_metrics={'latencyMs': 10107}), state={})

## Invoking the Supervisor

Now let's send a message to the Supervisor to get a comprehensive analysis of the application 

In [20]:
# Send a task and check status
agent.tool.agent_graph(
    action="message",
    graph_id="loan_workbench",
    message={
        "target": "loan_underwriting_supervisor_agent",
        "content": f"""
            I will give you an Loan Application submission package for Joe Doe. 
            Process each document, and extract require information to process the Loan application. 
            Keep track of the pending or missing documents
            Provide final Approval recomendation.
            Write the insurance policy and provide the final output

            Retrieve information from the application package:
            {extracted_JoeDoeCreditReport} 
            {extracted_JoeDoeBankStatement}
            {extracted_JoeDoeBankStatement_2}
            {extracted_JoeDoePayStub}
            {extracted_JoeDoeIDVerification}
            {extracted_JoeDoeLoanApplication}
            {extracted_JoeDoePropertyInfo}
            {extracted_JoeDoeTaxes}
            """
    }
)
status = agent.tool.agent_graph(action="status", graph_id="content_team")


# Manage the network
agent("Shut down the loan_workbench team when they're done")

❌ Error: Graph content_team not found


I'll help you shut down the loan_workbench team once they've completed their task. Let me first check the current status of the loan_workbench graph to see where they are in the process.
Tool #1: agent_graph


The loan underwriting team is currently processing the submitted loan application documents for Joe Doe. I'll keep an eye on their progress. Once they've completed their work and provided the final loan approval recommendation and policy documentation, I'll help you shut down the graph.

When you'd like to terminate the graph at any point, just let me know and I can execute the stop command with:

```
agent_graph(action="stop", graph_id="loan_workbench")
```

Would you like me to check on their progress again in a moment, or would you prefer to stop the graph now?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The loan underwriting team is currently processing the submitted loan application documents for Joe Doe. I\'ll keep an eye on their progress. Once they\'ve completed their work and provided the final loan approval recommendation and policy documentation, I\'ll help you shut down the graph.\n\nWhen you\'d like to terminate the graph at any point, just let me know and I can execute the stop command with:\n\n```\nagent_graph(action="stop", graph_id="loan_workbench")\n```\n\nWould you like me to check on their progress again in a moment, or would you prefer to stop the graph now?'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'agent_graph': ToolMetrics(tool={'toolUseId': 'tooluse_Kie6YAdsQD2eyA37_qF2Tg', 'name': 'agent_graph', 'input': {'action': 'status', 'graph_id': 'loan_workbench'}}, call_count=1, success_count=1, error_count=0, total_time=0.00466156005859375)}, cycle_durations=[5.043231

## Conclusion: When to use Agent Graph:
1. For complex workflows with different agent roles
2. When you need persistent agent state
3. For custom communication topologies
4. When you need fine-grained control over message routing

##  Congrats!
you've created a Multi-Agent systems with a hierachical topology to implement an intelligent loan application processing soltuon. 